In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn import linear_model, metrics, model_selection
import random

In [2]:
data = pd.read_csv('download.csv')
dateFormatter = "%d-%b-%y"
for i in range(data.shape[0]):
    data.iloc[i,0] = datetime.strptime(data.iloc[i,0], dateFormatter)
#新增漲跌的label
data['rof'] = 0
for i in range(data.shape[0]):
    if data.iloc[i,2]>data.iloc[i-1,2]:
        data.loc[i,'rof'] = 1
data.head()

,Date,Open Price,Close Price,High Price,Low Price,Volume,rof
0,2009-01-02 00:00:00,902.99,931.80,934.73,899.35,4048270080,0
1,2009-01-05 00:00:00,929.17,927.45,936.63,919.53,5413910016,0
2,2009-01-06 00:00:00,931.17,934.70,943.85,927.28,5392620032,1
3,2009-01-07 00:00:00,927.45,906.65,927.45,902.37,4704940032,0
4,2009-01-08 00:00:00,905.73,909.73,910.00,896.81,4991549952,1


In [3]:
train = data[data['Date']<=pd.datetime(2018,1,1)]
test = data[data['Date']>=pd.datetime(2018,1,1)]
x_train = train.loc[:, train.columns != 'rof']
y_train = train['rof']
x_test = test.loc[:, test.columns != 'rof']
y_test = test['rof']
x_train = x_train.drop('Date',axis = 1)
x_test = x_test.drop('Date',axis = 1)

y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)

X_train_std = scaler.transform(x_train)
X_test_std = scaler.transform(x_test)


C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


## Logistic Regression

In [5]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression().fit(X_train_std, y_train['rof'].values)
test_preds = lr.predict(X_test_std)
test_acc = metrics.accuracy_score(y_test['rof'].values, test_preds)
print(test_acc)

0.8253968253968254


C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## SVM

In [6]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
  
penalty = 10
#penalty = np.linspace(start=0.05, stop=10, num=200)

#for c in penalty:
svm = SVC(C=penalty, kernel="linear", probability=True)
svm.fit(X_train_std, y_train['rof'].values)
acc_rate = accuracy_score(y_test['rof'].values, svm.predict(X_test_std)) * 100
print("Penalty = %.2f, Accuracy = %.2f %%" % (penalty, acc_rate))

Penalty = 10.00, Accuracy = 82.94 %


## NN

In [7]:
import pandas as pd
from keras import models, layers, optimizers, regularizers
import numpy as np
import random
from sklearn import model_selection, preprocessing
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [19]:
hidden_units = 10  # how many neurons in the hidden layer
activation = 'relu'  # activation function for hidden layer
l2 = 0.001           # regularization - how much we penalize large parameter values
learning_rate = 0.1  # how big our steps are in gradient descent
epochs = 20          # how many epochs to train for
batch_size = 32      # how many samples to use for each gradient descent update

In [20]:
# create a sequential model
model = models.Sequential()

# add the hidden layer
model.add(layers.Dense(input_dim=5,
                       units=hidden_units, 
                       activation=activation))

# add the output layer
model.add(layers.Dense(input_dim=hidden_units,
                       units=1,
                       activation='sigmoid'))

# define our loss function and optimizer
model.compile(loss='binary_crossentropy',
              # Adam is a kind of gradient descent
              optimizer=optimizers.Adam(lr=learning_rate),
              metrics=['accuracy'])

In [21]:
class_weight = {
    0: 1.,
    1: 160 / 302
}

In [22]:
def train_and_evaluate(model, x_train, y_train, x_test, y_test, n=20):
    train_accs = []
    test_accs = []
    with tqdm(total=n) as progress_bar:
        for _ in range(n):
            model.fit(
                x_train, 
                y_train, 
                epochs=epochs, 
                batch_size=batch_size, 
                class_weight=class_weight, 
                verbose=False)
            train_accs.append(model.evaluate(x_train, y_train, batch_size=32, verbose=False)[1])
            test_accs.append(model.evaluate(x_test, y_test, batch_size=32, verbose=False)[1])
            progress_bar.update()
    print('Avgerage Training Accuracy: %s' % np.average(train_accs))
    print('Avgerage Testing Accuracy: %s' % np.average(test_accs))
    return train_accs, test_accs

In [23]:
_, test_accs = train_and_evaluate(model, X_train_std, y_train['rof'].values, X_test_std, y_test['rof'].values)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:24<00:00,  1.19s/it]


Avgerage Training Accuracy: 0.4540636044508998
Avgerage Testing Accuracy: 0.47619047902879263
